In [2]:
import json

# 入力ファイル（あなたの元データ）のパス
INPUT_JSON = "snack/test_snack_yesno.json"
# 出力ファイル（LLaVA OneVision 用 JSONL）のパス
OUTPUT_JSONL = "snack/test_snack_yesno.jsonl"

def convert_entry(entry):
    """
    元の entry を LLaVA OneVision 用フォーマットに変換する。
    """
    return {
        # サンプル識別子として id を source に
        "source": entry["id"],
        # 画像はリストに入れる
        "images": [entry["image"]],
        # 動画は今回は使わないので空リスト
        "videos": [],
        # system_prompt は未使用なので None
        "system_prompt": None,
        # 会話テキストをユーザ→アシスタントの順で並べたリストに
        "conversations": [
            entry["conversations"][0]["value"],
            entry["conversations"][1]["value"],
        ]
    }

def main():
    # 元データを読み込む
    with open(INPUT_JSON, "r", encoding="utf-8") as f:
        data = json.load(f)

    # JSONL に書き出す
    with open(OUTPUT_JSONL, "w", encoding="utf-8") as fout:
        for entry in data:
            new_entry = convert_entry(entry)
            fout.write(json.dumps(new_entry, ensure_ascii=False) + "\n")

    print(f"Converted {len(data)} entries to {OUTPUT_JSONL}")

if __name__ == "__main__":
    main()


Converted 1927 entries to snack/test_snack_yesno.jsonl


llava one visonようにtestを変換

In [ ]:
import json

def transform(input_path: str, output_path: str):
    # 1. JSON を読み込む
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    result = []
    for full_key, entry in data.items():
        # 2. "test/" プレフィックスを除去して、id と image に使う相対パスを得る
        rel_path = full_key.split('test/', 1)[-1]

        # 3. カテゴリ名はパスの最初の要素
        category = rel_path.split('/', 1)[0]

        # 4. bboxes の中に座標リストが空でない要素があれば "1"，なければ "0"
        has_bbox = any(box.get('bbox') for box in entry.get('bboxes', []))
        gpt_value = "1" if has_bbox else "0"

        # 5. human のプロンプト
        human_value = (
            "<image>\n"
            f"This is an image of {category}. "
            f"Does this {category} in the image have any defects?"
        )

        # 6. 結果オブジェクトを組み立て
        obj = {
            "id": rel_path,
            "image": rel_path,
            "conversations": [
                {"from": "human", "value": human_value},
                {"from": "gpt",   "value": gpt_value}
            ]
        }
        result.append(obj)

    # 7. 新しい JSON を書き出す
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)


if __name__ == '__main__':
    # 実際に変換したいファイル名を指定してください
    transform('input.json', 'output.json')


プロンプトを固定

Please answer yes or noは必要かどうか比較したい

In [2]:
#プロンプトを固定
import json

# 入出力ファイル名を適宜書き換えてください
input_path = "/home/okada/vlm/lmms-finetune/jsons/test0_finetune_data_onevision.jsonl"
output_path = "test0_yesno.jsonl"

# 固定プロンプト
fixed_prompt = "<image>\nIs this wood grain visually appealing to you? Answer with yes or no."

with open(input_path, "r", encoding="utf-8") as infile, open(output_path, "w", encoding="utf-8") as outfile:
    for line in infile:
        data = json.loads(line)
        # プロンプトの固定
        if "conversations" in data and isinstance(data["conversations"], list) and len(data["conversations"]) >= 2:
            data["conversations"][0] = fixed_prompt

            # 応答を "0" または "1" に変換（Yes → 0, No → 1）
            answer = data["conversations"][1].strip().lower()
            if answer.startswith("yes"):
                data["conversations"][1] = "0"
            elif answer.startswith("no"):
                data["conversations"][1] = "1"

        outfile.write(json.dumps(data, ensure_ascii=False) + "\n")



データシャッフル

In [3]:
import json
import random

with open("train0_yesno.jsonl", "r", encoding="utf-8") as f:
    lines = f.readlines()

random.shuffle(lines)  # ランダムに並べ替え

with open("train0_yesno_random.jsonl", "w", encoding="utf-8") as f:
    for line in lines:
        f.write(line)